# Analise do Algorítmo para cálculo das médias em Dev Médias
## Início

Este projeto visa ter o cálculo, a partir de notas de entrada, juntamente com todos os pesos de todas as notas, e uma média visada, encontrar as melhores notas que um aluno deve tirar para conseguir a tal média esperada. 
Inicialmente, este problema será separado em problemáticas, a fim de compreendê-lo melhor. 


## 0 - Estruturação do problema
Cada nota será separada em uma classe, com peso e valor da nota. 
1

### 0.0 - Importando bibliotecas

In [1]:
from typing import Tuple, List, Dict
import abc
import random 
import time
import math

### 0.1 - Definição das variáveis/ constantes

In [2]:
dominio_de_notas = list(map(lambda x: x/2, range(0,21)))


### 0.2 - Regras de negócio
### 0.2.1 - Nota
- O valor da nota pode ser None, mas o peso não;
- Cada nota deve pertencer ao domínio `dominio_de_notas`;
- O peso deve ser menor que 1

In [3]:
class Nota:
    peso: float # representa o peso da nota na matéria
    nota: float = None # representa o valor da nota

    def __init__(self, peso: float = None, nota: float = None):
        nota_valida, msg = self.valida_nota(nota)
        if(not nota_valida): 
            raise Exception(msg)
        self.nota = nota
        
        peso_valido, msg = self.valida_peso(peso)
        if(not peso_valido):
            raise Exception(msg)
        self.peso = peso
        
    @staticmethod
    def valida_nota(nota: int) -> Tuple[bool, str]:
        if(type(nota) not in [float, int, None]): 
            return (False, f"Nota {nota} deve ser um número")
        if(nota not in dominio_de_notas): 
            return (False, f"Nota {nota} deve estar entre 0 e 10, variando de 0.5 em 0.5")
        return (True, '')
        
    @staticmethod
    def valida_peso(peso: int) -> Tuple[bool, str]:
        if(peso is None): 
            return (False, f"Peso {peso} não pode ser nulo")
        elif(type(peso) != float): 
            return (False, f"Peso {peso} deve ser um número")
        elif (peso < 0): 
            return (False, f"Peso {peso} não pode ser menor que 0")
        elif(peso > 1): 
            return (False, f"Peso {peso} não pode ser maior que 1")
        return (True, '')
    
    def __str__(self):
        return f"(Valor: {self.nota}, Peso: {self.peso})"

Testando classe `Nota`

In [4]:
# Sucesso
nota = Nota(peso=0.5, nota=10)
print("Teste - Sucesso")
print(nota.nota == 10)
print(nota.peso == 0.5)

# Nota deve ser um número
print("\nTeste - Nota deve ser um número")
try:
    nota = Nota(peso=0.5, nota='10')
    print(False)
except Exception as e:
    print(e.args[0] == "Nota 10 deve ser um número")
    
# Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
print("\nTeste - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5")
try:
    nota = Nota(peso=0.5, nota=11)
    print(False)
except Exception as e:
    print(e.args[0] == "Nota 11 deve estar entre 0 e 10, variando de 0.5 em 0.5")
    
# Peso não pode ser nulo
print("\nTeste - Peso não pode ser nulo")
try:
    nota = Nota(nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso None não pode ser nulo")
    
# Peso deve ser um número
print("\nTeste - Peso deve ser um número")
try:
    nota = Nota(peso='0.5', nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso 0.5 deve ser um número")
    
# Peso não pode ser menor que 0
print("\nTeste - Peso não pode ser menor que 0")
try:
    nota = Nota(peso=-0.5, nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso -0.5 não pode ser menor que 0")
    
# Peso não pode ser maior que 1
print("\nTeste - Peso não pode ser maior que 1")
try:
    nota = Nota(peso=1.5, nota=10)
    print(False)
except Exception as e:
    print(e.args[0] == "Peso 1.5 não pode ser maior que 1")



Teste - Sucesso
True
True

Teste - Nota deve ser um número
True

Teste - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
True

Teste - Peso não pode ser nulo
True

Teste - Peso deve ser um número
True

Teste - Peso não pode ser menor que 0
True

Teste - Peso não pode ser maior que 1
True


#### 0.2.2 - Utils
Funções que serão utilizadas com frequência


In [5]:
class Utils:
    
    @staticmethod
    def print_lista_de_notas(l: List[Nota]):
        if(len(l) == 0):
            print("[]")
            return
        print("[", end=' ')
        for idx in range(len(l)-1):
            print(l[idx], end=', ')
        print(l[-1], end=' ]')

    @staticmethod
    def media_aritimetica(l: List[float]) -> float:
        return sum(l)/len(l)
        
    @staticmethod
    def media(l: List[Nota]) -> float:
        if sum(map(lambda x: x.peso, l)) != 1:
            raise Exception("A soma dos pesos deve ser 1")
        return sum(map(lambda x: x.nota * x.peso, l))

    @staticmethod
    def desvio_padrao(l: List[float]) -> float:
        media = Utils.media_aritimetica(l)
        return (sum(map(lambda x: (x - media)**2, l))/(len(l)-1))**(1/2)
    
    @staticmethod
    def distancia_entre_notas(l: List[float], distancia_min: float) -> bool:
        media = Utils.media_aritimetica(l)
        return all(map(lambda x: abs(x - media) <= distancia_min, l))

Testando funções de `Utils`

In [6]:
# Media - Teste 1
print("\nTeste - Media - Teste 1")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=10)]
print(Utils.media(notas) == 10)

# Media - Teste 2
print("\nTeste - Media - Teste 2")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=0)]
print(Utils.media(notas) == 5)

# Media - Teste 3
print("\nTeste - Media - Teste 3")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=5)]
print(Utils.media(notas) == 7.5)

# Media - Teste 4
print("\nTeste - Media - Teste 4")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.3, nota=5), Nota(peso=0.2, nota=0)]
print(Utils.media(notas) == 6.5)

# Media - Teste do Erro
print("\nTeste - Media - Teste do Erro")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=5), Nota(peso=0.5, nota=0), Nota(peso=0.5, nota=0)]
try:
    Utils.media(notas)
    print(False)
except Exception as e:
    print(e.args[0] == "A soma dos pesos deve ser 1")

# Desvio Padrão - Teste 1
print("\nTeste - Desvio Padrão - Teste 1")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=10)]
print(Utils.desvio_padrao([x.nota for x in notas]) == 0)

# Desvio Padrão - Teste 2
print("\nTeste - Desvio Padrão - Teste 2")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=0)]
print(7.0 < Utils.desvio_padrao([x.nota for x in notas]) < 7.1)

# Desvio Padrão - Teste 3
print("\nTeste - Desvio Padrão - Teste 3")
notas = [Nota(peso=0.25, nota=10), Nota(peso=0.3, nota=5), Nota(peso=0.2, nota=6), Nota(peso=0.25, nota=6)]
print(2.2 < Utils.desvio_padrao([x.nota for x in notas]) < 2.3)

# Distancia entre notas - Teste 1
print("\nTeste - Distancia entre notas - Teste 1")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=10)]
print(Utils.distancia_entre_notas([x.nota for x in notas], 0.5) == True)

# Distancia entre notas - Teste 2
print("\nTeste - Distancia entre notas - Teste 2")
notas = [Nota(peso=0.5, nota=10), Nota(peso=0.5, nota=0)]
print(Utils.distancia_entre_notas([x.nota for x in notas], 0.5) == False)

# Distancia entre notas - Teste 3
print("\nTeste - Distancia entre notas - Teste 3")
notas = [Nota(peso=0.25, nota=7), Nota(peso=0.3, nota=5), Nota(peso=0.2, nota=6), Nota(peso=0.25, nota=6)]
print(Utils.distancia_entre_notas([x.nota for x in notas], 3) == True)


Teste - Media - Teste 1
True

Teste - Media - Teste 2
True

Teste - Media - Teste 3
True

Teste - Media - Teste 4
True

Teste - Media - Teste do Erro
True

Teste - Desvio Padrão - Teste 1
True

Teste - Desvio Padrão - Teste 2
True

Teste - Desvio Padrão - Teste 3
True

Teste - Distancia entre notas - Teste 1
True

Teste - Distancia entre notas - Teste 2
True

Teste - Distancia entre notas - Teste 3
True


#### 0.2.3 - Solucionador
Interface para classes que resolverão os problemas abaixo

In [7]:
class Solucionador(abc.ABC):
    
    @abc.abstractmethod
    def algoritmo(self, notas_que_tenho: List[Tuple[int, int]], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        pass
    
    @abc.abstractmethod
    def teste_algoritmo(self, notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        pass

#### 0.2.4 - Observações de regras de negócio do algoritmo

- *exemplo explicativo:* se o aluno tirar entre 4.96 - 4.99 de média, ele terá média final 5.0;
- se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração;
- se for impossível do aluno tirar uma média (*ex:* quer tirar média 10, mas já tirou 0 em alguma média), o algoritmo deve notificar;

## 1. Primeiro Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1 e P2, quero saber para quais notas P3 e P4 o aluno conseguirá média 6.
Facilitadores:
1. tamanho de notas fixo
2. poucas notas
3. nota possível de ser determinada

In [8]:
class PrimeiroCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Tuple[int, int]], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        for i in range(len(dominio_de_notas)):
            for j in range(len(dominio_de_notas)):
                if(Utils.media(
                    [Nota(nota[0], nota[1]) for nota in notas_que_tenho] + \
                    [
                        Nota(peso=peso_de_notas_que_quero[0], nota=dominio_de_notas[i]),
                        Nota(peso=peso_de_notas_que_quero[1], nota=dominio_de_notas[j])
                    ]
                )) == media_desejada:
                    notas_possiveis.append((dominio_de_notas[i], dominio_de_notas[j]))
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = PrimeiroCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print(f"Para as notas {notas_que_tenho}, pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        for notas in notas_possiveis:
            print(f"{notas[0]} e {notas[1]}")

In [9]:
# Teste 1
print("\nPrimeiroCaso - Teste 1")
P1 = Nota(peso=0.2, nota=6.0)
P2 = Nota(peso=0.2, nota=5.0)
P3_peso = 0.4
P4_peso = 0.2
media_desejada = 6

PrimeiroCaso.teste_algoritmo(notas_que_tenho=[(P1.peso, P1.nota), (P2.peso, P2.nota)], peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=5.0),
    Nota(peso=P4_peso, nota=8.0)
]) == media_desejada)
)

# Teste 2
print("\nPrimeiroCaso - Teste 2")
P1 = Nota(peso=0.2, nota=7.0)
P2 = Nota(peso=0.2, nota=7.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

PrimeiroCaso.teste_algoritmo(notas_que_tenho=[(P1.peso, P1.nota), (P2.peso, P2.nota)], peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=6.5),
    Nota(peso=P4_peso, nota=7.5)
])==media_desejada)
)




PrimeiroCaso - Teste 1
Para as notas [(0.2, 6.0), (0.2, 5.0)], pesos [0.4, 0.2] e média 6 as notas possíveis são:
4.5 e 10.0
5.0 e 9.0
5.5 e 8.0
7.0 e 5.0
7.5 e 4.0
8.0 e 3.0
9.5 e 0.0

False

PrimeiroCaso - Teste 2
Para as notas [(0.2, 7.0), (0.2, 7.0)], pesos [0.3, 0.3] e média 7 as notas possíveis são:
4.0 e 10.0
4.5 e 9.5
5.0 e 9.0
5.5 e 8.5
6.0 e 8.0
6.5 e 7.5
7.0 e 7.0
8.0 e 6.0
8.5 e 5.5
9.0 e 5.0
9.5 e 4.5

True


**Observação**: Poucas combinações de `notas_que_tenho` e `media_desejada` calculam o que quero...

## 2. Segundo Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

In [10]:
class SegundoCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        for i in range(len(dominio_de_notas)):
            for j in range(len(dominio_de_notas)):
                for k in range(len(dominio_de_notas)):
                    for l in range(len(dominio_de_notas)):
                        for m in range(len(dominio_de_notas)):
                            if(Utils.media(
                                notas_que_tenho + \
                                [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_de_notas[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_de_notas[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_de_notas[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_de_notas[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_de_notas[m])
                                ]
                            ) == media_desejada):
                                notas_possiveis.append((dominio_de_notas[i], dominio_de_notas[j], dominio_de_notas[k], dominio_de_notas[l], dominio_de_notas[m]))
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = SegundoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

Para 5 notas o tempo está de mais ou menos 27s ...
Talvez escolhendo apenas 3 notas possa ajudar ...

Outro ponto a se observar é que também tem casos em que não existem notas possíveis de se determinar as médias...

## 3. Terceiro Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis

In [11]:
class TerceiroCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        for i in range(len(dominio_de_notas)):
            for j in range(len(dominio_de_notas)):
                for k in range(len(dominio_de_notas)):
                    for l in range(len(dominio_de_notas)):
                        for m in range(len(dominio_de_notas)):
                            if(Utils.media(
                                notas_que_tenho + \
                                [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_de_notas[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_de_notas[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_de_notas[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_de_notas[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_de_notas[m])
                                ]
                            ) == media_desejada):
                                notas_possiveis.append((dominio_de_notas[i], dominio_de_notas[j], dominio_de_notas[k], dominio_de_notas[l], dominio_de_notas[m]))
                            if(len(notas_possiveis) == 3): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = TerceiroCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [12]:
# Teste 1
print("\nTerceiroCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.2*0.4, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.2*0.4, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

TerceiroCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)


TerceiroCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08000000000000002), (Valor: 6.0, Peso: 0.08000000000000002) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (0.0, 0.0, 8.0, 9.0, 10.0), (0.0, 0.0, 8.5, 9.0, 9.5), (0.0, 0.0, 9.0, 9.0, 9.0) ]

Agora o algorítmo está com 0.2 s de duração...

Porém,  ele escolheu notas de grande esforço para tirar.

## 4. Quarto Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis
2. a ordem da permutação será aleatória

In [13]:
class QuartoCaso(Solucionador):
    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_i = dominio_de_notas.copy()
        random.shuffle(dominio_i)
        for i in range(len(dominio_de_notas)):
            dominio_j = dominio_de_notas.copy()
            random.shuffle(dominio_j)
            for j in range(len(dominio_de_notas)):
                dominio_k = dominio_de_notas.copy()
                random.shuffle(dominio_k)                
                for k in range(len(dominio_de_notas)):
                    dominio_l = dominio_de_notas.copy()
                    random.shuffle(dominio_l)
                    for l in range(len(dominio_de_notas)):
                        dominio_m = dominio_de_notas.copy()
                        random.shuffle(dominio_m)
                        for m in range(len(dominio_de_notas)):
                            if(Utils.media(
                                notas_que_tenho + \
                                [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_i[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_j[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_k[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_l[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_m[m])
                                ]
                            ) == media_desejada):
                                notas_possiveis.append((dominio_i[i], dominio_j[j], dominio_k[k], dominio_l[l], dominio_m[m]))
                            if(len(notas_possiveis) == 3): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = QuartoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [14]:
# Teste 1
print("QuartoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.2*0.4, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.2*0.4, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

QuartoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

QuartoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08000000000000002), (Valor: 6.0, Peso: 0.08000000000000002) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (1.0, 7.5, 7.0, 2.0, 10.0), (1.0, 7.5, 7.0, 8.0, 6.0), (1.0, 7.5, 7.0, 6.5, 7.0) ]

1. Se não tem notas que sua combinação dêem a média pedida, o tempo de espera é enorme
2. Se for possível uma combinação de notas, mesmo assim tem chance das notas não possuirem um bom "mínimo esforço"
3. O programa exibe o valor do peso de forma estranha: `Peso: 0.08000000000000002`

## 5. Quinto Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis
2. a ordem da permutação será aleatória
3. o algorítmo deve escolher notas com, no máximo, 3 de desvio padrão
4. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.

In [15]:
class QuintoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DESV_PAD=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_i = dominio_de_notas.copy()
        random.shuffle(dominio_i)
        for i in range(len(dominio_de_notas)):
            dominio_j = dominio_de_notas.copy()
            random.shuffle(dominio_j)
            for j in range(len(dominio_de_notas)):
                dominio_k = dominio_de_notas.copy()
                random.shuffle(dominio_k)                
                for k in range(len(dominio_de_notas)):
                    dominio_l = dominio_de_notas.copy()
                    random.shuffle(dominio_l)
                    for l in range(len(dominio_de_notas)):
                        dominio_m = dominio_de_notas.copy()
                        random.shuffle(dominio_m)
                        for m in range(len(dominio_de_notas)):
                            possiveis_notas_que_quero = [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_i[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_j[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_k[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_l[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_m[m])
                                ]
                            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
                            media = Utils.media(todas_as_notas)
                            if(abs(media - media_desejada) <= 0.04):
                                if(Utils.desvio_padrao([nota.nota for nota in possiveis_notas_que_quero]) < QuintoCaso.MIN_DESV_PAD):
                                    notas_possiveis.append((dominio_i[i], dominio_j[j], dominio_k[k], dominio_l[l], dominio_m[m]))
                            if(len(notas_possiveis) == QuintoCaso.NOTAS_TOTAIS): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = QuintoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [16]:
# Teste 1
print("\nQuintoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

QuintoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)


QuintoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (8.0, 5.5, 3.0, 9.5, 5.5), (8.0, 5.5, 3.0, 6.0, 8.0), (8.0, 5.5, 3.0, 9.0, 6.0) ]

1. O resultado melhorou, mas ainda as notas não parecem ser tão boas a fim se serem consideradas como "esforço mínimo"
2. talvez a ideia de "desvio padrão" não seja tão boa, se tentará criar um algorítmo que verifica se existem quaisquer valores que se distanciam de um valor fixo da média entre eles

## 6. Sexto Caso
Supondo que uma matéria tenha P1, P2, P3 e P4, sendo que já foram dadas as notas da P1, T1 e T2, quero saber para quais notas P2, P3, P4, T3 e T4 o aluno conseguirá média 7.
Facilitadores:
1. tamanho de notas fixo
2. nota possível de ser determinada

Adicionais:
1. o programa parará de rodar ao escolher 3 notas possíveis
2. a ordem da permutação será aleatória
3. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
4. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.

In [17]:
class SextoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DIST=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_i = dominio_de_notas.copy()
        random.shuffle(dominio_i)
        for i in range(len(dominio_de_notas)):
            dominio_j = dominio_de_notas.copy()
            random.shuffle(dominio_j)
            for j in range(len(dominio_de_notas)):
                dominio_k = dominio_de_notas.copy()
                random.shuffle(dominio_k)                
                for k in range(len(dominio_de_notas)):
                    dominio_l = dominio_de_notas.copy()
                    random.shuffle(dominio_l)
                    for l in range(len(dominio_de_notas)):
                        dominio_m = dominio_de_notas.copy()
                        random.shuffle(dominio_m)
                        for m in range(len(dominio_de_notas)):
                            possiveis_notas_que_quero = [
                                    Nota(peso=peso_de_notas_que_quero[0], nota=dominio_i[i]),
                                    Nota(peso=peso_de_notas_que_quero[1], nota=dominio_j[j]),
                                    Nota(peso=peso_de_notas_que_quero[2], nota=dominio_k[k]),
                                    Nota(peso=peso_de_notas_que_quero[3], nota=dominio_l[l]),
                                    Nota(peso=peso_de_notas_que_quero[4], nota=dominio_m[m])
                                ]
                            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
                            media = Utils.media(todas_as_notas)
                            if(abs(media - media_desejada) <= 0.04):
                                if(Utils.distancia_entre_notas([nota.nota for nota in possiveis_notas_que_quero], SextoCaso.MIN_DIST)):
                                    notas_possiveis.append((dominio_i[i], dominio_j[j], dominio_k[k], dominio_l[l], dominio_m[m]))
                            if(len(notas_possiveis) == SextoCaso.NOTAS_TOTAIS): return notas_possiveis
        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = SextoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)

In [18]:
# Teste 1
print("\nSextoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

SextoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)


SextoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (9.0, 5.5, 4.0, 3.0, 8.5), (9.0, 5.5, 4.0, 9.0, 4.5), (9.0, 5.5, 4.0, 5.5, 6.5) ]

Melhorou bastante a escolha entre as notas. Se tentará resolver os outros pontos que não foram explorados até agora.

## 7. Sétimo Caso
Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.
Facilitadores:
1. nota possível de ser determinada

Novidade:
1. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer tamanho de notas de input
2. o programa parará de rodar ao escolher 3 notas possíveis
3. a ordem da permutação será aleatória

Deve ser adicionado:
1. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
2. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.

In [19]:
class SetimoCaso(Solucionador):
    NOTAS_TOTAIS=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_de_possiveis_notas = list()
        for _ in range(len(peso_de_notas_que_quero)):
            dominio_de_uma_nota = dominio_de_notas.copy()
            random.shuffle(dominio_de_uma_nota)
            dominio_de_possiveis_notas.append(dominio_de_uma_nota)
        idx_possiveis_notas = [0 for _ in range(len(peso_de_notas_que_quero))]
        
        while(idx_possiveis_notas != [19 for _ in range(len(peso_de_notas_que_quero))]):
            possiveis_notas_que_quero = [Nota(peso=peso_de_notas_que_quero[i], nota=dominio_de_possiveis_notas[i][idx_possiveis_notas[i]]) for i in range(len(peso_de_notas_que_quero))]
            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
            media = Utils.media(todas_as_notas)

            if(media == media_desejada):
                notas_possiveis.append(tuple([nota.nota for nota in possiveis_notas_que_quero]))
            
            if(len(notas_possiveis) == SetimoCaso.NOTAS_TOTAIS): return notas_possiveis

            idx_possiveis_notas[-1] += 1
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas)-1):
                if(idx_possiveis_notas_invertido[idx]) == 20:
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = SetimoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)
        print        

In [20]:
# Teste 1
print("\nSetimoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

SetimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2_peso, nota=2.5),
    Nota(peso=T3_peso, nota=2),
    Nota(peso=P3_peso, nota=6),
    Nota(peso=T4_peso, nota=9),
    Nota(peso=P4_peso, nota=9)
]

print("\n",abs(Utils.media(notas) - media_desejada)<= 0.04)

# Teste 2
print("\nSetimoCaso - Teste 2")
P1 = Nota(peso=0.2, nota=7.0)
P2 = Nota(peso=0.2, nota=7.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

l_notas_que_tenho = [P1, P2]

SetimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=9.0),
    Nota(peso=P4_peso, nota=5.0)
]) - media_desejada) <= 0.04)
)




SetimoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (3.0, 4.5, 8.5, 6.0, 6.5), (3.0, 4.5, 8.5, 4.5, 7.5), (3.0, 4.5, 8.5, 1.5, 9.5) ]
 True

SetimoCaso - Teste 2
Para as notas:
[ (Valor: 7.0, Peso: 0.2), (Valor: 7.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 7 as notas possíveis são:
[ (8.0, 6.0), (9.0, 5.0), (5.0, 9.0) ]
True


Funcionou. Agora será implementado as melhorias até agora feitas

## 8. Oitavo Caso
Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.

Facilitadores:
1. nota possível de ser determinada

Novidade:
1. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer tamanho de notas de input
2. o programa parará de rodar ao escolher 3 notas possíveis
3. a ordem da permutação será aleatória
4. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
5. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [21]:
class OitavoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DIST=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_de_possiveis_notas = list()
        for _ in range(len(peso_de_notas_que_quero)):
            dominio_de_uma_nota = dominio_de_notas.copy()
            random.shuffle(dominio_de_uma_nota)
            dominio_de_possiveis_notas.append(dominio_de_uma_nota)
        idx_possiveis_notas = [0 for _ in range(len(peso_de_notas_que_quero))]
        
        while(idx_possiveis_notas != [19 for _ in range(len(peso_de_notas_que_quero))]):
            possiveis_notas_que_quero = [Nota(peso=peso_de_notas_que_quero[i], nota=dominio_de_possiveis_notas[i][idx_possiveis_notas[i]]) for i in range(len(peso_de_notas_que_quero))]
            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
            media = Utils.media(todas_as_notas)

            if(abs(media - media_desejada) <= 0.04):
                if(Utils.distancia_entre_notas([nota.nota for nota in possiveis_notas_que_quero], OitavoCaso.MIN_DIST)):
                    notas_possiveis.append(tuple([nota.nota for nota in possiveis_notas_que_quero]))
            
            if(len(notas_possiveis) == OitavoCaso.NOTAS_TOTAIS): return notas_possiveis

            idx_possiveis_notas[-1] += 1
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas)-1):
                if(idx_possiveis_notas_invertido[idx]) == 20:
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        return notas_possiveis
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = OitavoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} as notas possíveis são:")
        Utils.print_lista_de_notas(notas_possiveis)
        print        

In [22]:
# Teste 1
print("\nOitavoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

OitavoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2_peso, nota=5),
    Nota(peso=T3_peso, nota=3),
    Nota(peso=P3_peso, nota=4.5),
    Nota(peso=T4_peso, nota=8.5),
    Nota(peso=P4_peso, nota=8.5)
]

print("\n",abs(Utils.media(notas)) - media_desejada <= 0.04)

# Teste 2
print("\nOitavoCaso - Teste 2")
P1 = Nota(peso=0.2, nota=6.0)
P2 = Nota(peso=0.2, nota=8.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

l_notas_que_tenho = [P1, P2]

OitavoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=9.0),
    Nota(peso=P4_peso, nota=5.0)
]) - media_desejada) <= 0.04)
)




OitavoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 as notas possíveis são:
[ (2.5, 6.0, 8.5, 2.5, 8.0), (2.5, 4.5, 8.5, 4.0, 8.0), (2.5, 4.5, 8.5, 3.5, 8.5) ]
 True

OitavoCaso - Teste 2
Para as notas:
[ (Valor: 6.0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 7 as notas possíveis são:
[ (6.0, 8.0), (6.5, 7.5), (5.5, 8.5) ]
True


## 9. Nono Caso

Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.

Facilitadores:
1. nota possível de ser determinada

O que deve ser implementado:
1. se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração;
2. se for impossível do aluno tirar uma média (*ex:* quer tirar média 10, mas já tirou 0 em alguma média), o algoritmo deve notificar;

Novidade:
1. o algoritmo deve retornar uma das 3 notas escolhidas, sendo esta a de menor desvio padrão 
2. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer tamanho de notas de input
3. o programa parará de rodar ao escolher 3 notas possíveis
4. a ordem da permutação será aleatória
5. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética entre elas
6. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [23]:
class NonoCaso(Solucionador):
    NOTAS_TOTAIS=3
    MIN_DIST=3

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list()
        dominio_de_possiveis_notas = list()
        for _ in range(len(peso_de_notas_que_quero)):
            dominio_de_uma_nota = dominio_de_notas.copy()
            random.shuffle(dominio_de_uma_nota)
            dominio_de_possiveis_notas.append(dominio_de_uma_nota)
        idx_possiveis_notas = [0 for _ in range(len(peso_de_notas_que_quero))]
        
        while(idx_possiveis_notas != [19 for _ in range(len(peso_de_notas_que_quero))]):
            possiveis_notas_que_quero = [Nota(peso=peso_de_notas_que_quero[i], nota=dominio_de_possiveis_notas[i][idx_possiveis_notas[i]]) for i in range(len(peso_de_notas_que_quero))]
            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero
            media = Utils.media(todas_as_notas)

            if(abs(media - media_desejada) <= 0.04):
                if(Utils.distancia_entre_notas([nota.nota for nota in possiveis_notas_que_quero], NonoCaso.MIN_DIST)):
                    notas_possiveis.append(tuple([nota.nota for nota in possiveis_notas_que_quero]))
            
            if(len(notas_possiveis) == NonoCaso.NOTAS_TOTAIS): 
                melhor_combinacao = notas_possiveis[0]
                for nota in range(1, len(notas_possiveis)):
                    if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                        melhor_combinacao = notas_possiveis[nota]
                return melhor_combinacao

            idx_possiveis_notas[-1] += 1
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas)-1):
                if(idx_possiveis_notas_invertido[idx]) == 20:
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        if(len(notas_possiveis) == 0):
            return []
        else:
            melhor_combinacao = notas_possiveis[0]
            for nota in range(1, len(notas_possiveis)):
                if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                    melhor_combinacao = notas_possiveis[nota]
            return melhor_combinacao
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = NonoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} uma combinação de notas possíveis é:")
        Utils.print_lista_de_notas(notas_possiveis)

In [24]:
# Teste 1
print("\nNonoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

NonoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2_peso, nota=8.5),
    Nota(peso=T3_peso, nota=5.5),
    Nota(peso=P3_peso, nota=7),
    Nota(peso=T4_peso, nota=5),
    Nota(peso=P4_peso, nota=4.5)
]

print("\n",(Utils.media(notas) - media_desejada) <= 0.04)

# Teste 2
print("\nNonoCaso - Teste 2")
P1 = Nota(peso=0.2, nota=6.0)
P2 = Nota(peso=0.2, nota=8.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

l_notas_que_tenho = [P1, P2]

NonoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=5.5),
    Nota(peso=P4_peso, nota=8.5)
]) - media_desejada) <= 0.04)
)




NonoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 uma combinação de notas possíveis é:
[ 6.0, 6.0, 4.0, 5.5, 8.5 ]
 True

NonoCaso - Teste 2
Para as notas:
[ (Valor: 6.0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 7 uma combinação de notas possíveis é:
[ 7.0, 7.0 ]
True


## 10. Décimo Caso

Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.

O que deve ser implementado:
1. se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração;

Novidade:
1. se não existir nenhuma combinação na qual o aluno atinja aquela média (mesmo médias maiores), o programa reportará
2. o algoritmo deve retornar uma das 3 notas escolhidas, sendo esta a de menor desvio padrão 
3. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer 3amanho de notas de input
4. o programa parará de rodar ao escolher 3 notas possíveis
5. a ordem da permutação será aleatória
6. o algorítmo deve escolher notas com, no máximo, 3 de distância entre as notas e a média aritimética 6ntre elas
7. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [25]:
class DecimoCaso(Solucionador):
    NOTAS_TOTAIS=3 # quantidade de notas que o programa escolherá e parará ao encontrá-los
    MIN_DIST=3  # menor distância entre notas escolhidas e a média aritimética entre elas
                # para que sejam escolhidas pelo algorítmo

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], peso_de_notas_que_quero: List[float], media_desejada: float) -> List[Nota]:
        notas_possiveis = list() # lista que conterá as notas possíveis de serem retornadas
        dominio_de_possiveis_notas = list() # lista que conterá o domínio de cada nota escolhida (futuramente serão embaralhados)
                                            # ex: [[0, 0.5, ..., 9.5, 10], [0, 0.5, ..., 9.5, 10], ...]

        if (Utils.media(notas_que_tenho + [Nota(peso=peso_de_notas_que_quero[i], nota=10) for i in range(len(peso_de_notas_que_quero))]) - media_desejada < 0):    # Se não for possível atingir tal nota, retornará uma lista vazia
                                # ex: se o aluno escolher média 10, e tirou 0 em alguma nota, esse "if" captará
            return []

        for _ in range(len(peso_de_notas_que_quero)):   # preenchimento da lista "dominio_de_possiveis_notas"
                                                        # com domínio de cada nota 
            dominio_de_uma_nota = dominio_de_notas.copy()
            random.shuffle(dominio_de_uma_nota)
            dominio_de_possiveis_notas.append(dominio_de_uma_nota)
        idx_possiveis_notas = [0 for _ in range(len(peso_de_notas_que_quero))]  # lista que conterá o index da vez de análse da função,
                                                                                # começando com [0,0,0, ...] com o tamanho dependendo 
                                                                                # da quantidade de notas que o programa quer calcular
        
        while(idx_possiveis_notas != [19 for _ in range(len(peso_de_notas_que_quero))]):    # rodará até encontrar 3 notas possíveis 
                                                                                            # ou acabar as notas
            possiveis_notas_que_quero = [
                Nota(
                peso=peso_de_notas_que_quero[i],
                nota=dominio_de_possiveis_notas[i][idx_possiveis_notas[i]]
                ) for i in range(len(peso_de_notas_que_quero))
            ]   # lista de notas que quero determinar, utilizando a lista de domínios de notas para montá-las
                # formando, assim, uma combinação de notas

            todas_as_notas = notas_que_tenho + possiveis_notas_que_quero    # junção das notas que tenho com combinação
                                                                            # previamente feita
            media = Utils.media(todas_as_notas) # cálculo da média desta iteração

            if(abs(media - media_desejada) <= 0.04): # verifica se a média varia de 0.04 em relação à média desejada
                if(Utils.distancia_entre_notas([nota.nota for nota in possiveis_notas_que_quero], DecimoCaso.MIN_DIST)):    # verifica se
                                                                                                                            # todas as 
                                                                                                                            # notas distam
                                                                                                                            # da média
                                                                                                                            # no máximo
                                                                                                                            # MIN_DIST

                    notas_possiveis.append(tuple([nota.nota for nota in possiveis_notas_que_quero]))
            
            if(len(notas_possiveis) == DecimoCaso.NOTAS_TOTAIS):    # faz o break do laço se encontra NOTAS_TOTAIS como tamanho da lista
                                                                    # de combinações de notas possíveis
                melhor_combinacao = notas_possiveis[0] 
                for nota in range(1, len(notas_possiveis)): # lógica para verificar quais das combinações escolhidas 
                                                            # de notas possui o menor desvio padrão
                    if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                        melhor_combinacao = notas_possiveis[nota]
                return melhor_combinacao

            idx_possiveis_notas[-1] += 1    # se não for possível parar o laço, adiciona 1 ao último index da lista de index
                                            # ex: [0, 0, 0, ..., 0, 0] --(+1)--> [0, 0, 0, ..., 0, 0]
                                            # ex2: [0, 0, 0, ..., 0, 19] --(+1)--> [0, 0, 0, ..., 0, 20] -> [0, 0, 0, ..., 1, 0]
                                            # ex3: [0, 0, 0, ..., 0, 19, 19] --(+1)--> [0, 0, 0, ..., 0, 19, 20] -> [0, 0, 0, ..., 0, 20, 0] ->
                                            # -> [0, 0, 0, ..., 1, 0, 0]
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas_invertido)-1):
                if(idx_possiveis_notas_invertido[idx] == 20):
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        if(len(notas_possiveis) == 0): # se não encontrou nenhuma nota
            return []
        else:   # se encontrou alguma(s) combinação(ões) de nota(s), faz o cálculo do desvio padrão e
                # retorna a combinação com menor desvio padrão
            melhor_combinacao = notas_possiveis[0]
            for nota in range(1, len(notas_possiveis)):
                if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                    melhor_combinacao = notas_possiveis[nota]
            return melhor_combinacao
    
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], peso_de_notas_que_quero: List[float], media_desejada: float) -> None:
        notas_possiveis = DecimoCaso.algoritmo(notas_que_tenho=notas_que_tenho, peso_de_notas_que_quero=peso_de_notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f", pesos {peso_de_notas_que_quero} e média {media_desejada} uma combinação de notas possíveis é:")
        Utils.print_lista_de_notas(notas_possiveis)

In [26]:
# Teste 1
print("\nDecimoCaso - Teste 1")
P1 = Nota(peso=0.2*0.6, nota=6.0)
T1 = Nota(peso=0.08, nota=6.0)

P2_peso = 0.2*0.6
T2 = Nota(peso=0.08, nota=6.0)

P3_peso = 0.3*0.6
T3_peso = 0.3*0.4

P4_peso = 0.3*0.6
T4_peso = 0.3*0.4

media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

DecimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P2_peso, T3_peso, P3_peso, T4_peso, P4_peso], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2_peso, nota=8.5),
    Nota(peso=T3_peso, nota=5.5),
    Nota(peso=P3_peso, nota=7),
    Nota(peso=T4_peso, nota=5),
    Nota(peso=P4_peso, nota=4.5)
]

print("\n",(Utils.media(notas) - media_desejada) <= 0.04)

# Teste 2
print("\nDecimoCaso - Teste 2")
P1 = Nota(peso=0.2, nota=6.0)
P2 = Nota(peso=0.2, nota=8.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 7

l_notas_que_tenho = [P1, P2]

DecimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3_peso, nota=5.5),
    Nota(peso=P4_peso, nota=8.5)
]) - media_desejada) <= 0.04)
)

# Teste 3
print("\nDecimoCaso - Teste 3")
P1 = Nota(peso=0.2, nota=0)
P2 = Nota(peso=0.2, nota=8.0)
P3_peso = 0.3
P4_peso = 0.3
media_desejada = 10

l_notas_que_tenho = [P1, P2]

DecimoCaso.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, peso_de_notas_que_quero=[P3_peso, P4_peso], media_desejada=media_desejada)


DecimoCaso - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ], pesos [0.12, 0.12, 0.18, 0.12, 0.18] e média 6 uma combinação de notas possíveis é:
[ 7.5, 6.5, 4.5, 5.5, 6.5 ]
 True

DecimoCaso - Teste 2
Para as notas:
[ (Valor: 6.0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 7 uma combinação de notas possíveis é:
[ 8.5, 5.5 ]
True

DecimoCaso - Teste 3
Para as notas:
[ (Valor: 0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ], pesos [0.3, 0.3] e média 10 uma combinação de notas possíveis é:
[]


Algumas alterações foram pedidas:
- transformar em variável: len(peso_de_notas_que_quero)
- passar pro algorítmo classe Nota, com nota.nota = None para as notas que não tenho
- fazer o descarte de domínio para as notas
  - atribui 10 a todas as notas menos uma $\rightarrow$ verifica seu valor mínimo
  - atribui 0 a todas as notas menos uma $\rightarrow$ verifica seu valor máximo 
- adicionar, dentro da classe "Nota", o domínio de tal nota, utilizando-o no algorítmo

## 11.1 Mudança na classe `Nota`

- mudando atributo `nota.nota` para `nota.valor`
- adicionando um atributo `nota.dominio_da_nota`, representando seu domínio individual
  - criação desta serve para:
    - reduzir individualmente o domínio de cada nota, fazendo com que se diminuam as repetições do laço de repetição
    - fazer uma maior dependência da classe `Nota`

In [27]:
class Nota:
    peso: float # representa o peso da nota na matéria
    valor: float = None # representa o valor da nota
    dominio_da_nota: List[float] # representa o domínio de notas possíveis para uma nota em específico
    DOMINIO_DE_NOTAS = list(map(lambda x: x/2, range(0,21)))

    def __init__(self, peso: float = None, valor: float = None):
        nota_valida, msg = self.valida_valor(valor)
        if(not nota_valida and valor != None): 
            raise Exception(msg)
        self.valor = valor
        
        peso_valido, msg = self.valida_peso(peso)
        if(not peso_valido):
            raise Exception(msg)
        self.peso = peso

        self.dominio_da_nota = self.DOMINIO_DE_NOTAS.copy()
        
    def randomiza_dominio(self) -> None:
        random.shuffle(self.dominio_da_nota)

    def limita_dominio(self, valor_minimo: float, valor_maximo: float) -> None:
        dominio_valido, msg = self.valida_limitacao_de_dominio(self.dominio_da_nota, valor_minimo, valor_maximo)
        if(not dominio_valido):
            raise Exception(msg)
        self.dominio_da_nota = [nota for nota in self.dominio_da_nota if nota >= valor_minimo and nota <= valor_maximo]

    @staticmethod
    def valida_limitacao_de_dominio(dominio_a_ser_limitado: List[float], valor_minimo: float, valor_maximo: float) -> Tuple[bool, str]:
        if(valor_maximo < valor_minimo):
            return (False, f"Valor mínimo {valor_minimo} deve ser menor que valor máximo {valor_maximo}")
        
        valor_minimo_valido, msg = Nota.valida_valor(valor_minimo)
        if (not valor_minimo_valido):
            return (valor_minimo_valido, msg)
        
        valor_maximo_valido, msg = Nota.valida_valor(valor_maximo)
        if (not valor_maximo_valido):
            return (valor_maximo_valido, msg)
        
        elif(len(dominio_a_ser_limitado) != len(Nota.DOMINIO_DE_NOTAS)):
            return (False, f"Domínio da nota já foi limitado")
        elif(dominio_a_ser_limitado != Nota.DOMINIO_DE_NOTAS):
            return (False, f"Domínio da nota já foi embaralhado")
        else:
            return (True, '')

    @staticmethod
    def valida_valor(valor: int) -> Tuple[bool, str]:
        if(type(valor) not in [float, int]): 
            return (False, f"Valor de nota {valor} deve ser um número")
        if(valor not in Nota.DOMINIO_DE_NOTAS): 
            return (False, f"Valor de nota {valor} deve estar entre 0 e 10, variando de 0.5 em 0.5")
        return (True, '')
        
    @staticmethod
    def valida_peso(peso: int) -> Tuple[bool, str]:
        if(peso is None): 
            return (False, f"Peso {peso} não pode ser nulo")
        elif(type(peso) != float): 
            return (False, f"Peso {peso} deve ser um número")
        elif (peso < 0): 
            return (False, f"Peso {peso} não pode ser menor que 0")
        elif(peso > 1): 
            return (False, f"Peso {peso} não pode ser maior que 1")
        return (True, '')



    def __str__(self):
        return f"(Valor: {self.valor}, Peso: {self.peso})"

Testando mudanças da classe `Nota`

In [28]:
quantidade_de_testes_da_classe_Nota_errados = 0

def valida_teste(real, previsto, quantidade_de_testes_da_classe_Nota_errados) -> int:
    if(real == previsto):
        print("Sucesso")
        return quantidade_de_testes_da_classe_Nota_errados
    else:
        print(f"Falha: \"{real}\" deveria ser \"{previsto}\"")
        return quantidade_de_testes_da_classe_Nota_errados + 1

# Sucesso
print("Teste - Sucesso")
try:
    nota = Nota(peso=0.5, valor=10)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.valor, 10, quantidade_de_testes_da_classe_Nota_errados)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.peso, 0.5, quantidade_de_testes_da_classe_Nota_errados)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.dominio_da_nota, Nota.DOMINIO_DE_NOTAS, quantidade_de_testes_da_classe_Nota_errados)
except Exception as e:
    print(f"Erro: {e.args[0]}")
    quantidade_de_testes_da_classe_Nota_errados += 1

# Possível valor 'None' para nota.valor
print("\nTeste - Possível valor 'None' para nota.valor")
try:
    nota = Nota(peso=0.5)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.valor, None, quantidade_de_testes_da_classe_Nota_errados)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.peso, 0.5, quantidade_de_testes_da_classe_Nota_errados)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.dominio_da_nota, Nota.DOMINIO_DE_NOTAS, quantidade_de_testes_da_classe_Nota_errados)
except:
    print(f"Erro: {e.args[0]}")
    quantidade_de_testes_da_classe_Nota_errados += 1

# Nota deve ser um número
print("\nTeste - Nota deve ser um número")
try:
    nota = Nota(peso=0.5, valor='10')
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Valor de nota 10 deve ser um número", quantidade_de_testes_da_classe_Nota_errados)
    
# Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
print("\nTeste - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5")
try:
    nota = Nota(peso=0.5, valor=11)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Valor de nota 11 deve estar entre 0 e 10, variando de 0.5 em 0.5", quantidade_de_testes_da_classe_Nota_errados)
    
# Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
print("\nTeste 2 - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5")
try:
    nota = Nota(peso=0.5, valor=-1)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Valor de nota -1 deve estar entre 0 e 10, variando de 0.5 em 0.5", quantidade_de_testes_da_classe_Nota_errados)
    
# Peso não pode ser nulo
print("\nTeste - Peso não pode ser nulo")
try:
    nota = Nota(valor=10)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Peso None não pode ser nulo", quantidade_de_testes_da_classe_Nota_errados)
    
# Peso deve ser um número
print("\nTeste - Peso deve ser um número")
try:
    nota = Nota(peso='0.5', valor=10)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Peso 0.5 deve ser um número", quantidade_de_testes_da_classe_Nota_errados)
    
# Peso não pode ser menor que 0
print("\nTeste - Peso não pode ser menor que 0")
try:
    nota = Nota(peso=-0.5, valor=10)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Peso -0.5 não pode ser menor que 0", quantidade_de_testes_da_classe_Nota_errados)

# Peso não pode ser maior que 1
print("\nTeste - Peso não pode ser maior que 1")
try:
    nota = Nota(peso=1.5, valor=10)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Peso 1.5 não pode ser maior que 1", quantidade_de_testes_da_classe_Nota_errados)

# Teste do método 'nota.randomiza_dominio'
print("\nTeste - Teste do método 'nota.randomiza_dominio'")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.randomiza_dominio()
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.dominio_da_nota != Nota.DOMINIO_DE_NOTAS, True, quantidade_de_testes_da_classe_Nota_errados)
    print("Domínio randomizado: ", nota.dominio_da_nota)
except Exception as e:
    print(f"Erro: {e.args[0]}")
    quantidade_de_testes_da_classe_Nota_errados += 1

# Teste do método 'nota.limita_dominio'
print("\nTeste - Teste do método 'nota.limita_dominio'")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(0, 5.0)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.dominio_da_nota == list(map(lambda x: x/2, range(0,11))), True, quantidade_de_testes_da_classe_Nota_errados)
    print("Domínio limitado: ", nota.dominio_da_nota)
except Exception as e:
    print(f"Erro: {e.args[0]}")
    quantidade_de_testes_da_classe_Nota_errados += 1

# Teste 2 do método 'nota.limita_dominio'
print("\nTeste 2 - Teste do método 'nota.limita_dominio'")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(4.5, 10)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.dominio_da_nota == list(map(lambda x: x/2, range(9,21))), True, quantidade_de_testes_da_classe_Nota_errados)
    print("Domínio limitado: ", nota.dominio_da_nota)
except Exception as e:
    print(f"Erro: {e.args[0]}")
    quantidade_de_testes_da_classe_Nota_errados += 1

# Teste 3 do método 'nota.limita_dominio'
print("\nTeste 3 - Teste do método 'nota.limita_dominio'")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(3.5, 8)
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(nota.dominio_da_nota == list(map(lambda x: x/2, range(7,17))), True, quantidade_de_testes_da_classe_Nota_errados)
    print("Domínio limitado: ", nota.dominio_da_nota)
except Exception as e:
    print(f"Erro: {e.args[0]}")
    quantidade_de_testes_da_classe_Nota_errados += 1

# Teste de falha do método 'nota.limita_dominio' - valor mínimo deve ser menor que valor máximo
print("\nTeste de falha do método 'nota.limita_dominio' -  valor mínimo deve ser menor que valor máximo")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(8, 3)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Valor mínimo 8 deve ser menor que valor máximo 3", quantidade_de_testes_da_classe_Nota_errados)

# Teste de falha do método 'nota.limita_dominio' - valor mínimo deve ser maior que 0
print("\nTeste de falha do método 'nota.limita_dominio' -  valor mínimo deve ser maior que 0")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(-1, 3)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Valor de nota -1 deve estar entre 0 e 10, variando de 0.5 em 0.5", quantidade_de_testes_da_classe_Nota_errados)

# Teste de falha do método 'nota.limita_dominio' - valor máximo deve ser menor que 10
print("\nTeste de falha do método 'nota.limita_dominio' -  valor máximo deve ser menor que 10")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(8, 11)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Valor de nota 11 deve estar entre 0 e 10, variando de 0.5 em 0.5", quantidade_de_testes_da_classe_Nota_errados)

# Teste de falha do método 'nota.limita_dominio' - domínio_de_notas já foi limitado anteriormente
print("\nTeste de falha do método 'nota.limita_dominio' - Domínio da nota já foi limitado")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.limita_dominio(8, 10)
    nota.limita_dominio(8, 10)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Domínio da nota já foi limitado", quantidade_de_testes_da_classe_Nota_errados)

# Teste de falha do método 'nota.limita_dominio' - domínio_de_notas já foi randomizado anteriormente
print("\nTeste de falha do método 'nota.limita_dominio' - Domínio da nota já foi embaralhado")
try:
    nota = Nota(peso=0.5, valor=10)
    nota.randomiza_dominio()
    nota.limita_dominio(8, 10)
    print("Erro não foi lançado")
    quantidade_de_testes_da_classe_Nota_errados += 1
except Exception as e:
    quantidade_de_testes_da_classe_Nota_errados = valida_teste(e.args[0], "Domínio da nota já foi embaralhado", quantidade_de_testes_da_classe_Nota_errados)




# Mostrando se há testes errados
if quantidade_de_testes_da_classe_Nota_errados != 0:
    print(f"---------- EXISTE(M) {quantidade_de_testes_da_classe_Nota_errados} TESTE(S) ERRADO(S) ----------")

Teste - Sucesso
Sucesso
Sucesso
Sucesso

Teste - Possível valor 'None' para nota.valor
Sucesso
Sucesso
Sucesso

Teste - Nota deve ser um número
Sucesso

Teste - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
Sucesso

Teste 2 - Nota deve estar entre 0 e 10, variando de 0.5 em 0.5
Sucesso

Teste - Peso não pode ser nulo
Sucesso

Teste - Peso deve ser um número
Sucesso

Teste - Peso não pode ser menor que 0
Sucesso

Teste - Peso não pode ser maior que 1
Sucesso

Teste - Teste do método 'nota.randomiza_dominio'
Sucesso
Domínio randomizado:  [1.0, 0.5, 7.5, 4.5, 9.0, 2.5, 3.5, 8.5, 0.0, 5.5, 10.0, 5.0, 4.0, 9.5, 8.0, 6.5, 1.5, 2.0, 6.0, 3.0, 7.0]

Teste - Teste do método 'nota.limita_dominio'
Sucesso
Domínio limitado:  [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

Teste 2 - Teste do método 'nota.limita_dominio'
Sucesso
Domínio limitado:  [4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]

Teste 3 - Teste do método 'nota.limita_dominio'
Sucesso
Domínio limitado:  

## 11.2 - Alterando Utils (para não quebrar os experimentos acima)



### Criação do cálculo da equação formada para o método `minimo_valor_no_dominio` e `máximo_valor_no_dominio`:

$$abs\left(\frac{\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}}n_{quero_i}\cdot p_{quero_i}}{\sum_{i=0}^{len_{tenho}}p_{tenho_i}+\sum_{i=0}^{len_{quero}}p_{quero_i}} - media\right) \le 0.04$$

$$abs\left(\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}}n_{quero_i}\cdot p_{quero_i} - média \right) \le 0.04$$

$$abs\left(\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}-1}n_{quero_i}\cdot p_{quero_i}+n_x\cdot p_x - média \right) \le 0.04$$

Se quero saber o valor mínimo desse $n_x$:

$$\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}-1}n_{quero_i}\cdot p_{quero_i}+n_{min}\cdot p_x = média - 0.04$$

$$\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}-1}10\cdot p_{quero_i}+n_{min}\cdot p_x = média - 0.04$$

$$n_{min}\cdot p_x = média - 0.04 - \sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i} - 10\sum_{i=0}^{len_{quero}-1} p_{quero_i}$$

$$\therefore n_{min} = \dfrac{média - 0.04 - \sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i} - 10\sum_{i=0}^{len_{quero}-1} p_{quero_i}}{p_x}$$

Quero o menor valor para o domínio total de notas possíveis. Portanto:

$$n_{min} = \dfrac{ceil\left(2\cdot \left(\dfrac{média - 0.04 - \sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i} - 10\sum_{i=0}^{len_{quero}-1} p_{quero_i}}{p_x}\right)\right)}{2} $$

Se quero saber o valor máximo desse $n_x$:

$$\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}-1}n_{quero_i}\cdot p_{quero_i}+n_{máx}\cdot p_x = média + 0.04$$

$$\sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}+\sum_{i=0}^{len_{quero}-1}0\cdot p_{quero_i}+n_{máx}\cdot p_x = média + 0.04$$

$$n_{máx}\cdot p_x =  média + 0.04 - \sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}$$

$$\therefore n_{máx} =  \dfrac{média + 0.04 - \sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}}{p_x} $$

Quero o maior valor para o domínio total de notas possíveis. Portanto:

$$n_{máx} =  \dfrac{floor\left(2\cdot \left(\dfrac{média + 0.04 - \sum_{i=0}^{len_{tenho}}n_{tenho_i}\cdot p_{tenho_i}}{p_x}\right)\right)}{2} $$


In [29]:
class Utils:
    
    @staticmethod
    def print_lista_de_notas(l: List[Nota]):
        if(len(l) == 0):
            print("[]")
            return
        print("[", end=' ')
        for idx in range(len(l)-1):
            print(l[idx], end=', ')
        print(l[-1], end=' ]')

    @staticmethod
    def media_aritimetica(l: List[Nota]) -> float:
        return sum([nota.valor for nota in l])/len(l)
        
    @staticmethod
    def media(l: List[Nota]) -> float:
        if sum(map(lambda x: x.peso, l)) != 1:
            raise Exception("A soma dos pesos deve ser 1")
        return sum(map(lambda x: x.valor * x.peso, l))

    @staticmethod
    def desvio_padrao(l: List[Nota]) -> float:
        media = Utils.media_aritimetica(l)
        return (sum(map(lambda x: (x.valor - media)**2, l))/(len(l)-1))**(1/2)
    
    @staticmethod
    def distancia_entre_notas(l: List[Nota], distancia_min: float) -> bool:
        media = Utils.media_aritimetica(l)
        return all(map(lambda x: abs(x.valor - media) <= distancia_min, l))
    
    @staticmethod
    def print_pesos_de_notas(l: List[Nota]):
        if(len(l) == 0):
            print("[]", end="")
            return
        print("[", end=' ')
        for idx in range(len(l)-1):
            print(l[idx].peso, end=', ')
        print(l[-1].peso, end=' ]')

    @staticmethod
    def minimo_valor_no_dominio(notas_que_tenho: List[Nota], notas_que_quero: List[Nota], peso_especifico: float, media_desejada: float) -> float:
        valor = math.ceil((2 / peso_especifico) * (media_desejada - 0.04 - sum([nota.peso*nota.valor for nota in notas_que_tenho]) - 10 * sum([nota.peso for nota in notas_que_quero])))/2
        if valor < Nota.DOMINIO_DE_NOTAS[0]: # 0
            return Nota.DOMINIO_DE_NOTAS[0]
        elif valor > Nota.DOMINIO_DE_NOTAS[-1]: # 10
            return -1
        return valor

    @staticmethod
    def maximo_valor_no_dominio(notas_que_tenho: List[Nota], notas_que_quero: List[Nota], peso_especifico: float, media_desejada: float) -> float:
        valor = math.floor((2 / peso_especifico) * (media_desejada + 0.04 - sum([nota.peso*nota.valor for nota in notas_que_tenho])))/2
        if valor > Nota.DOMINIO_DE_NOTAS[-1]: # 10
            return Nota.DOMINIO_DE_NOTAS[-1]
        return valor

## 11.3 Caso 11

Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.

O que deve ser implementado:
1. fazer o descarte de domínio para as notas
     - atribui 10 a todas as notas menos uma $\rightarrow$ verifica seu valor mínimo
     - atribui 0 a todas as notas menos uma $\rightarrow$ verifica seu valor máximo 
2. se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração
3. não fazer a troca do valor das notas sempre rodando a lista de notas (trocar o valor da nota no momento em que se analisam os idx) 

```console
                              for idx, nota in enumerate(notas_que_quero):
                                             nota.valor = nota.dominio[idx_possiveis_notas[idx]]
```

4. melhorar ideia de crescimento da lista de idx


Novidade:
1. passar pro algorítmo classe Nota, com nota.nota = None para as notas que não tenho
2. adicionar, dentro da classe "Nota", o domínio de tal nota, utilizando-o no algorítmo
3. transformar em variável: len(peso_de_notas_que_quero)
4. o algorítmo deve escolher notas com, no máximo, 2 de distância entre as notas e a média aritimética entre elas
5. se não existir nenhuma combinação na qual o aluno atinja aquela média (mesmo médias maiores), o programa reportará
6. o algoritmo deve retornar uma das 3 notas escolhidas, sendo esta a de menor desvio padrão 
7. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer 3amanho de notas de input
8. o programa parará de rodar ao escolher 3 notas possíveis
9. a ordem da permutação será aleatória
10. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [30]:
class Caso11(Solucionador):
    NOTAS_TOTAIS=3 # quantidade de notas que o programa escolherá e parará ao encontrá-los
    MIN_DIST=2  # menor distância entre notas escolhidas e a média aritimética entre elas
                # para que sejam escolhidas pelo algorítmo

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], notas_que_quero: List[Nota], media_desejada: float) -> List[Nota]:
        # variável que representa o tamanho da lista `notas_que_quero`
        tamanho_notas_que_quero = len(notas_que_quero)

        # variável que representa o tamanho da lista `notas_que_tenho`
        tamanho_notas_que_tenho = len(notas_que_tenho)

        # lista que conterá as notas possíveis de serem retornadas
        notas_possiveis = list()
        
        # Se não for possível atingir tal nota, retornará uma lista vazia
        # ex: se o aluno escolher média 10, e tirou 0 em alguma nota, esse "if" captará
        # obs: Nota.DOMINIO_DE_NOTAS = 10
        if (Utils.media(notas_que_tenho + [Nota(peso=notas_que_quero[i].peso, valor=Nota.DOMINIO_DE_NOTAS[-1]) for i in range(tamanho_notas_que_quero)]) - media_desejada < 0):
            return []

        # embaralhamento dos dominios das notas que quero
        for nota in notas_que_quero:
            nota.randomiza_dominio()

        # lista que conterá o index da vez de análse da função, começando com [0,0,0, ...] 
        # com o tamanho dependendo da quantidade de notas que o programa quer calcular
        idx_possiveis_notas = [0 for _ in range(tamanho_notas_que_quero)]  
        
        # rodará até encontrar 3 notas possíveis ou acabar as notas 
        # obs: len(Nota.DOMINIO_DE_NOTAS) - 2 = 19
        while(idx_possiveis_notas != [len(Nota.DOMINIO_DE_NOTAS)-2 for _ in range(tamanho_notas_que_quero)]): 

            # lista de notas que quero determinar, utilizando a lista de domínios
            # de notas para montá-las formando, assim, uma combinação de notas
            for idx, nota in enumerate(notas_que_quero):
                nota.valor = nota.dominio_da_nota[idx_possiveis_notas[idx]]

            # junção das notas que tenho com combinação previamente feita
            todas_as_notas = notas_que_tenho + notas_que_quero    
            
            # cálculo da média desta iteração
            media = Utils.media(todas_as_notas)

            # verifica se a média varia de 0.04 em relação à média desejada
            if(abs(media - media_desejada) <= 0.04): 

                # verifica se todas as notas distam da média no máximo MIN_DIST
                if(Utils.distancia_entre_notas(notas_que_quero, Caso11.MIN_DIST)):    
                    notas_possiveis.append(tuple(notas_que_quero))
            
            # faz o break do laço se encontra NOTAS_TOTAIS como
            # tamanho da lista de combinações de notas possíveis
            if(len(notas_possiveis) == Caso11.NOTAS_TOTAIS):    
                melhor_combinacao = notas_possiveis[0] 

                # lógica para verificar quais das combinações escolhidas 
                # de notas possui o menor desvio padrão
                for nota in range(1, len(notas_possiveis)): 
                    if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                        melhor_combinacao = notas_possiveis[nota]
                return melhor_combinacao

            # se não for possível parar o laço, adiciona 1 ao último index da lista de index
            # ex: [0, 0, 0, ..., 0, 0] --(+1)--> [0, 0, 0, ..., 0, 0]
            # ex2: [0, 0, 0, ..., 0, 19] --(+1)--> [0, 0, 0, ..., 0, 20] -> [0, 0, 0, ..., 1, 0]
            # ex3: [0, 0, 0, ..., 0, 19, 19] --(+1)--> [0, 0, 0, ..., 0, 19, 20] -> [0, 0, 0, ..., 0, 20, 0] ->
            # -> [0, 0, 0, ..., 1, 0, 0]
            idx_possiveis_notas[-1] += 1   
            idx_possiveis_notas_invertido = idx_possiveis_notas.copy()[::-1]
            for idx in range(0, len(idx_possiveis_notas_invertido)-1):
                if(idx_possiveis_notas_invertido[idx] == 20):
                    idx_possiveis_notas_invertido[idx] = 0
                    idx_possiveis_notas_invertido[idx+1] += 1
                else:
                    break
                
            idx_possiveis_notas = idx_possiveis_notas_invertido[::-1]

        # se não encontrou nenhuma nota, retorna uma lista vazia
        if(len(notas_possiveis) == 0):
            return []
        
        # se encontrou alguma(s) combinação(ões) de nota(s), faz o cálculo
        # do desvio padrão e retorna a combinação com menor desvio padrão
        else:  
            melhor_combinacao = notas_possiveis[0]
            for nota in range(1, len(notas_possiveis)):
                if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                    melhor_combinacao = notas_possiveis[nota]
            return melhor_combinacao
    
    # função que exibirá, pelos inputs passados, a lista de combinações de notas possíveis escolhidas pelo algorítmo
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], notas_que_quero: List[Nota], media_desejada: float) -> None:
        notas_possiveis = Caso11.algoritmo(notas_que_tenho=notas_que_tenho, notas_que_quero=notas_que_quero, media_desejada=media_desejada)
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f" pesos:")
        Utils.print_pesos_de_notas(notas_que_quero)
        print(f", e média {media_desejada} uma combinação de notas possíveis é:")
        Utils.print_lista_de_notas(notas_possiveis)

In [31]:
# Teste 1
print("\nCaso11 - Teste 1")
P1 = Nota(peso=0.2*0.6, valor=6.0)
T1 = Nota(peso=0.08, valor=6.0)
T2 = Nota(peso=0.08, valor=6.0)

P2 = Nota(peso=0.2*0.6, valor=None)
P3 = Nota(peso=0.3*0.6, valor=None)
T3 = Nota(peso=0.3*0.4, valor=None)
P4 = Nota(peso=0.3*0.6, valor=None)
T4 = Nota(peso=0.3*0.4, valor=None)



media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

Caso11.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P2, T3, P3, T4, P4], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2.peso, valor=6.5),
    Nota(peso=T3.peso, valor=8),
    Nota(peso=P3.peso, valor=5),
    Nota(peso=T4.peso, valor=6),
    Nota(peso=P4.peso, valor=5.5)
]

print("\n",(Utils.media(notas) - media_desejada) <= 0.04)

# Teste 2
print("\nCaso11 - Teste 2")
P1 = Nota(peso=0.2, valor=6.0)
P2 = Nota(peso=0.2, valor=8.0)

P3 = Nota(peso=0.3, valor = None)
P4 = Nota(peso=0.3, valor = None)

media_desejada = 7

l_notas_que_tenho = [P1, P2]

Caso11.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P3, P4], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3.peso, valor=7.5),
    Nota(peso=P4.peso, valor=6.5)
]) - media_desejada) <= 0.04)
)

# Teste 3
print("\nCaso11 - Teste 3")
P1 = Nota(peso=0.2, valor=0)
P2 = Nota(peso=0.2, valor=8.0)
P3 = Nota(peso=0.3)
P4 = Nota(peso=0.3)
media_desejada = 10

l_notas_que_tenho = [P1, P2]

Caso11.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P3, P4], media_desejada=media_desejada)


Caso11 - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ] pesos:
[ 0.12, 0.12, 0.18, 0.12, 0.18 ], e média 6 uma combinação de notas possíveis é:
[ (Valor: 7.0, Peso: 0.12), (Valor: 6.5, Peso: 0.12), (Valor: 7.5, Peso: 0.18), (Valor: 4.0, Peso: 0.12), (Valor: 5.0, Peso: 0.18) ]
 True

Caso11 - Teste 2
Para as notas:
[ (Valor: 6.0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ] pesos:
[ 0.3, 0.3 ], e média 7 uma combinação de notas possíveis é:
[ (Valor: 9.0, Peso: 0.3), (Valor: 5.0, Peso: 0.3) ]
True

Caso11 - Teste 3
Para as notas:
[ (Valor: 0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ] pesos:
[ 0.3, 0.3 ], e média 10 uma combinação de notas possíveis é:
[]


Verificou-se duas possibilidades:
- para distância mínima = 3:
  - o algorítmo é rápido o suficiente
  - mas tem vezes que descobre notas não tão boas assim
- para distância mínima = 2:
  - o algorítmo descobre notas boas
  - mas tem vezes que ele é muito lento

## 12 Caso 12

Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.

O que deve ser implementado:
1. fazer o descarte de domínio para as notas
     - atribui 10 a todas as notas menos uma $\rightarrow$ verifica seu valor mínimo
     - atribui 0 a todas as notas menos uma $\rightarrow$ verifica seu valor máximo 
2. se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração



Novidade:
1. melhorar ideia de crescimento da lista de idx
2. adicionando tempo de resposta ao rodar algorítmo na exibição do teste
3. passar pro algorítmo classe Nota, com nota.nota = None para as notas que não tenho
4. adicionar, dentro da classe "Nota", o domínio de tal nota, utilizando-o no algorítmo
5. transformar em variável: len(peso_de_notas_que_quero)
6. o algorítmo deve escolher notas com, no máximo, 2 de distância entre as notas e a média aritimética entre elas
7. se não existir nenhuma combinação na qual o aluno atinja aquela média (mesmo médias maiores), o programa reportará
8. o algoritmo deve retornar uma das 3 notas escolhidas, sendo esta a de menor desvio padrão 
9. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer 3amanho de notas de input
10. o programa parará de rodar ao escolher 3 notas possíveis
11. a ordem da permutação será aleatória
12. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [32]:
class Caso12(Solucionador):
    NOTAS_TOTAIS=3 # quantidade de notas que o programa escolherá e parará ao encontrá-los
    MIN_DIST=2  # menor distância entre notas escolhidas e a média aritimética entre elas
                # para que sejam escolhidas pelo algorítmo

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], notas_que_quero: List[Nota], media_desejada: float) -> List[Nota]:
        # variável que representa o tamanho da lista `notas_que_quero`
        tamanho_notas_que_quero = len(notas_que_quero)

        # variável que representa o tamanho da lista `notas_que_tenho`
        tamanho_notas_que_tenho = len(notas_que_tenho)

        # lista que conterá as notas possíveis de serem retornadas
        notas_possiveis = list()
        
        # Se não for possível atingir tal nota, retornará uma lista vazia
        # ex: se o aluno escolher média 10, e tirou 0 em alguma nota, esse "if" captará
        # obs: Nota.DOMINIO_DE_NOTAS[-1] = 10
        if (Utils.media(notas_que_tenho + [Nota(peso=notas_que_quero[i].peso, valor=Nota.DOMINIO_DE_NOTAS[-1]) for i in range(tamanho_notas_que_quero)]) - media_desejada < 0):
            return []

        # embaralhamento dos dominios das notas que quero
        for nota in notas_que_quero:
            nota.randomiza_dominio()

        # lista que conterá o index da vez de análse da função, começando com [0,0,0, ...] 
        # com o tamanho dependendo da quantidade de notas que o programa quer calcular
        idx_possiveis_notas = [0 for _ in range(tamanho_notas_que_quero)]  
        
        # lista de notas que quero determinar, utilizando a lista de domínios
        # de notas para montá-las formando, assim, uma combinação de notas
        for idx, nota in enumerate(notas_que_quero):
            nota.valor = nota.dominio_da_nota[idx_possiveis_notas[idx]]

        # rodará até encontrar 3 notas possíveis ou acabar as notas 
        # verifica se chegou na iteração da última nota
        while(notas_que_quero[-1].valor != notas_que_quero[-1].dominio_da_nota[-1]): 

            # junção das notas que tenho com combinação previamente feita
            todas_as_notas = notas_que_tenho + notas_que_quero    
            
            # cálculo da média desta iteração
            media = Utils.media(todas_as_notas)

            # verifica se a média varia de 0.04 em relação à média desejada
            if(abs(media - media_desejada) <= 0.04): 

                # verifica se todas as notas distam da média no máximo MIN_DIST
                if(Utils.distancia_entre_notas(notas_que_quero, Caso12.MIN_DIST)):    
                    notas_possiveis.append(tuple(notas_que_quero))
            
            # faz o break do laço se encontra NOTAS_TOTAIS como
            # tamanho da lista de combinações de notas possíveis
            if(len(notas_possiveis) == Caso12.NOTAS_TOTAIS):    
                melhor_combinacao = notas_possiveis[0] 

                # lógica para verificar quais das combinações escolhidas 
                # de notas possui o menor desvio padrão
                for nota in range(1, len(notas_possiveis)): 
                    if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                        melhor_combinacao = notas_possiveis[nota]
                return melhor_combinacao

            # se não for possível parar o laço, adiciona 1 ao último index da lista de index
            # ex: [0, 0, 0, ..., 0, 0] --(+1)--> [0, 0, 0, ..., 0, 0]
            # ex2: [0, 0, 0, ..., 0, 19] --(+1)--> [0, 0, 0, ..., 0, 20] -> [0, 0, 0, ..., 1, 0]
            # ex3: [0, 0, 0, ..., 0, 19, 19] --(+1)--> [0, 0, 0, ..., 0, 19, 20] -> [0, 0, 0, ..., 0, 20, 0] ->
            # -> [0, 0, 0, ..., 1, 0, 0]
            idx_possiveis_notas[-1] += 1   
            
            for idx in list(range(1, len(idx_possiveis_notas)))[::-1]:
                if(idx_possiveis_notas[idx] == len(Nota.DOMINIO_DE_NOTAS)-1):
                    # zera o valor do index da nota dessa iteração
                    idx_possiveis_notas[idx] = 0

                    # alterando o valor da nota dessa iteração para o primeiro valor do domínio
                    notas_que_quero[idx].valor = notas_que_quero[idx].dominio_da_nota[idx_possiveis_notas[idx]]

                    # aumentando o index da nota seguinte
                    idx_possiveis_notas[idx-1] += 1
                else:
                    # alterando valor da nota dessa iteração
                    notas_que_quero[idx].valor = notas_que_quero[idx].dominio_da_nota[idx_possiveis_notas[idx]]
                    break
            

        # se não encontrou nenhuma nota, retorna uma lista vazia
        if(len(notas_possiveis) == 0):
            return []
        
        # se encontrou alguma(s) combinação(ões) de nota(s), faz o cálculo
        # do desvio padrão e retorna a combinação com menor desvio padrão
        else:  
            melhor_combinacao = notas_possiveis[0]
            for nota in range(1, len(notas_possiveis)):
                if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                    melhor_combinacao = notas_possiveis[nota]
            return melhor_combinacao
    
    # função que exibirá, pelos inputs passados, a lista de combinações de notas possíveis escolhidas pelo algorítmo
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], notas_que_quero: List[Nota], media_desejada: float) -> None:
        t_inicial = time.time()
        notas_possiveis = Caso12.algoritmo(notas_que_tenho=notas_que_tenho, notas_que_quero=notas_que_quero, media_desejada=media_desejada)
        t_final = time.time()

        t_exec = t_final - t_inicial
        
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f" pesos:")
        Utils.print_pesos_de_notas(notas_que_quero)
        print(f", e média {media_desejada} uma combinação de notas possíveis é:")
        Utils.print_lista_de_notas(notas_possiveis)
        print(f"\nO algorítmo demorou {t_exec:.5f} segundos para executar.")

In [34]:
# Teste 1
print("\nCaso12 - Teste 1")
P1 = Nota(peso=0.2*0.6, valor=6.0)
T1 = Nota(peso=0.08, valor=6.0)
T2 = Nota(peso=0.08, valor=6.0)

P2 = Nota(peso=0.2*0.6, valor=None)
P3 = Nota(peso=0.3*0.6, valor=None)
T3 = Nota(peso=0.3*0.4, valor=None)
P4 = Nota(peso=0.3*0.6, valor=None)
T4 = Nota(peso=0.3*0.4, valor=None)



media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

Caso12.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P2, T3, P3, T4, P4], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2.peso, valor=6.5),
    Nota(peso=T3.peso, valor=8),
    Nota(peso=P3.peso, valor=5),
    Nota(peso=T4.peso, valor=6),
    Nota(peso=P4.peso, valor=5.5)
]

print("\n",(Utils.media(notas) - media_desejada) <= 0.04)

# Teste 2
print("\nCaso12 - Teste 2")
P1 = Nota(peso=0.2, valor=6.0)
P2 = Nota(peso=0.2, valor=8.0)

P3 = Nota(peso=0.3, valor = None)
P4 = Nota(peso=0.3, valor = None)

media_desejada = 7

l_notas_que_tenho = [P1, P2]

Caso12.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P3, P4], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3.peso, valor=7.5),
    Nota(peso=P4.peso, valor=6.5)
]) - media_desejada) <= 0.04)
)

# Teste 3
print("\nCaso12 - Teste 3")
P1 = Nota(peso=0.2, valor=0)
P2 = Nota(peso=0.2, valor=8.0)
P3 = Nota(peso=0.3)
P4 = Nota(peso=0.3)
media_desejada = 10

l_notas_que_tenho = [P1, P2]

Caso12.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P3, P4], media_desejada=media_desejada)


Caso12 - Teste 1
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ] pesos:
[ 0.12, 0.12, 0.18, 0.12, 0.18 ], e média 6 uma combinação de notas possíveis é:
[ (Valor: 5.0, Peso: 0.12), (Valor: 6.5, Peso: 0.12), (Valor: 4.5, Peso: 0.18), (Valor: 6.0, Peso: 0.12), (Valor: 8.0, Peso: 0.18) ]
O algorítmo demorou 0.13601 segundos para executar.

 True

Caso12 - Teste 2
Para as notas:
[ (Valor: 6.0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ] pesos:
[ 0.3, 0.3 ], e média 7 uma combinação de notas possíveis é:
[ (Valor: 5.5, Peso: 0.3), (Valor: 8.5, Peso: 0.3) ]
O algorítmo demorou 0.00000 segundos para executar.

True

Caso12 - Teste 3
Para as notas:
[ (Valor: 0, Peso: 0.2), (Valor: 8.0, Peso: 0.2) ] pesos:
[ 0.3, 0.3 ], e média 10 uma combinação de notas possíveis é:
[]

O algorítmo demorou 0.00000 segundos para executar.


## 13 Caso 13

Supondo que tenha duas matérias, tal que elas não possuam os mesmos tamanhos de notas necessárias, quero saber para quais notas um aluno que tirou certas notas iniciais conseguirá tirar uma certa média pedida.

O que deve ser implementado:
1. se não existir uma combinação de na qual o aluno consegue tirar a média certinha, o algoritmo deve responder com alguma alteração



Novidade:
1. se o valor mínimo/ máximo das possíveis notas forem `0` e `10`, respectivamente, reduzir domínimo para `MIN_DIST-10-MIN_DIST`
2. fazer o descarte de domínio para as notas
     - atribui 10 a todas as notas menos uma $\rightarrow$ verifica seu valor mínimo
     - atribui 0 a todas as notas menos uma $\rightarrow$ verifica seu valor máximo 
     - por conta disso foi criado na classe `Utils`, definida pela segunda vez, com os métodos `minimo_valor_no_dominio` e `maximo_valor_no_dominio`
3. melhorar ideia de crescimento da lista de idx
4. adicionando tempo de resposta ao rodar algorítmo na exibição do teste
5. passar pro algorítmo classe Nota, com nota.nota = None para as notas que não tenho
6. adicionar, dentro da classe "Nota", o domínio de tal nota, utilizando-o no algorítmo
7. transformar em variável: len(peso_de_notas_que_quero)
8. o algorítmo deve escolher notas com, no máximo, 2 de distância entre as notas e a média aritimética entre elas
9. se não existir nenhuma combinação na qual o aluno atinja aquela média (mesmo médias maiores), o programa reportará
10. o algoritmo deve retornar uma das 3 notas escolhidas, sendo esta a de menor desvio padrão 
11. o programa deve aceitar qualquer tamanho de notas que devem ser determinadas, assim como qualquer 3amanho de notas de input
12. o programa parará de rodar ao escolher 3 notas possíveis
13. a ordem da permutação será aleatória
14. a média pode variar do valor escolhido para `0.04` abaixo do esperado (para aceitar aredondamentos). *Ex*: se a média desejada for `6.0`, a combinação aceita pode ter média final entre `5.96` e `6.0`.


In [35]:
class Caso13(Solucionador):
    NOTAS_TOTAIS=3 # quantidade de notas que o programa escolherá e parará ao encontrá-los
    MIN_DIST=3  # menor distância entre notas escolhidas e a média aritimética entre elas
                # para que sejam escolhidas pelo algorítmo

    @staticmethod
    def algoritmo(notas_que_tenho: List[Nota], notas_que_quero: List[Nota], media_desejada: float) -> List[Nota]:
        # variável que representa o tamanho da lista `notas_que_quero`
        tamanho_notas_que_quero = len(notas_que_quero)

        # variável que representa o tamanho da lista `notas_que_tenho`
        tamanho_notas_que_tenho = len(notas_que_tenho)

        # lista que conterá as notas possíveis de serem retornadas
        notas_possiveis = list()
        
        # Se não for possível atingir tal nota, retornará uma lista vazia
        # ex: se o aluno escolher média 10, e tirou 0 em alguma nota, esse "if" captará
        # obs: Nota.DOMINIO_DE_NOTAS[-1] = 10
        if (Utils.media(notas_que_tenho + [Nota(peso=notas_que_quero[i].peso, valor=Nota.DOMINIO_DE_NOTAS[-1]) for i in range(tamanho_notas_que_quero)]) - media_desejada < 0):
            return []

        #limitando o domínio de cada nota
        for idx, nota in enumerate(notas_que_quero):

            # seleciona o mínimo valor de cada nota para que seja possível calcular uma média válida
            valor_minimo = Utils.minimo_valor_no_dominio(notas_que_tenho=notas_que_tenho, notas_que_quero=notas_que_quero[:idx] + notas_que_quero[idx+1:], peso_especifico=nota.peso, media_desejada=media_desejada)
            
            # se `valor_minimo` for igual a -1, significa que não é possível atingir a média desejada
            if(valor_minimo == -1):
                return []

            # seleciona um máximo valor de cada nota para que seja possível calcular uma média válida
            valor_maximo = Utils.maximo_valor_no_dominio(notas_que_tenho=notas_que_tenho, notas_que_quero=notas_que_quero[:idx] + notas_que_quero[idx+1:], peso_especifico=nota.peso, media_desejada=media_desejada)

            # limitando o domínio da nota
            nota.limita_dominio(valor_minimo, valor_maximo)

        # embaralhamento dos dominios das notas que quero
        for nota in notas_que_quero:
            nota.randomiza_dominio()
            print(nota.dominio_da_nota)

        # lista que conterá o index da vez de análse da função, começando com [0,0,0, ...] 
        # com o tamanho dependendo da quantidade de notas que o programa quer calcular
        idx_possiveis_notas = [0 for _ in range(tamanho_notas_que_quero)]  
        
        # lista de notas que quero determinar, utilizando a lista de domínios
        # de notas para montá-las formando, assim, uma combinação de notas
        for idx, nota in enumerate(notas_que_quero):
            nota.valor = nota.dominio_da_nota[idx_possiveis_notas[idx]]

        # variável que representa que todas as notas foram verificadas
        todas_as_notas_verificadas = False

        # rodará até encontrar 3 notas possíveis ou acabar as notas 
        while(not todas_as_notas_verificadas): 
            
            # verifica se chegou a iteração da última nota
            if(notas_que_quero[-1].valor == notas_que_quero[-1].dominio_da_nota[-1]):
                todas_as_notas_verificadas = True

            # junção das notas que tenho com combinação previamente feita
            todas_as_notas = notas_que_tenho + notas_que_quero    
            
            # cálculo da média desta iteração
            media = Utils.media(todas_as_notas)

            # verifica se a média varia de 0.04 em relação à média desejada
            if(abs(media - media_desejada) <= 0.04): 

                # verifica se todas as notas distam da média no máximo MIN_DIST
                if(Utils.distancia_entre_notas(notas_que_quero, Caso13.MIN_DIST)):    
                    notas_possiveis.append(tuple(notas_que_quero))
            
            # faz o break do laço se encontra NOTAS_TOTAIS como
            # tamanho da lista de combinações de notas possíveis
            if(len(notas_possiveis) == Caso13.NOTAS_TOTAIS):    
                melhor_combinacao = notas_possiveis[0] 

                # lógica para verificar quais das combinações escolhidas 
                # de notas possui o menor desvio padrão
                for nota in range(1, len(notas_possiveis)): 
                    if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                        melhor_combinacao = notas_possiveis[nota]
                return melhor_combinacao

            # se não for possível parar o laço, adiciona 1 ao último index da lista de index
            # ex: [0, 0, 0, ..., 0, 0] --(+1)--> [0, 0, 0, ..., 0, 0]
            # ex2: [0, 0, 0, ..., 0, 19] --(+1)--> [0, 0, 0, ..., 0, 20] -> [0, 0, 0, ..., 1, 0]
            # ex3: [0, 0, 0, ..., 0, 19, 19] --(+1)--> [0, 0, 0, ..., 0, 19, 20] -> [0, 0, 0, ..., 0, 20, 0] ->
            # -> [0, 0, 0, ..., 1, 0, 0]
            idx_possiveis_notas[-1] += 1   
            
            for idx in list(range(1, len(idx_possiveis_notas)))[::-1]:
                if(idx_possiveis_notas[idx] == len(notas_que_quero[idx].dominio_da_nota)-1):
                    # zera o valor do index da nota dessa iteração
                    idx_possiveis_notas[idx] = 0

                    # alterando o valor da nota dessa iteração para o primeiro valor do domínio
                    notas_que_quero[idx].valor = notas_que_quero[idx].dominio_da_nota[idx_possiveis_notas[idx]]

                    # aumentando o index da nota seguinte
                    idx_possiveis_notas[idx-1] += 1
                else:
                    # alterando valor da nota dessa iteração
                    notas_que_quero[idx].valor = notas_que_quero[idx].dominio_da_nota[idx_possiveis_notas[idx]]
                    break
            
        # se não encontrou nenhuma nota, retorna uma lista vazia
        if(len(notas_possiveis) == 0):
            return []
        
        # se encontrou alguma(s) combinação(ões) de nota(s), faz o cálculo
        # do desvio padrão e retorna a combinação com menor desvio padrão
        else:  
            melhor_combinacao = notas_possiveis[0]
            for nota in range(1, len(notas_possiveis)):
                if(Utils.desvio_padrao(notas_possiveis[nota]) - media_desejada < Utils.desvio_padrao(melhor_combinacao) - media_desejada):
                    melhor_combinacao = notas_possiveis[nota]
            return melhor_combinacao
    
    # função que exibirá, pelos inputs passados, a lista de combinações de notas possíveis escolhidas pelo algorítmo
    @staticmethod
    def teste_algoritmo(notas_que_tenho: Dict[float, float], notas_que_quero: List[Nota], media_desejada: float) -> None:
        t_inicial = time.time()
        notas_possiveis = Caso13.algoritmo(notas_que_tenho=notas_que_tenho, notas_que_quero=notas_que_quero, media_desejada=media_desejada)
        t_final = time.time()

        t_exec = t_final - t_inicial
        
        print("Para as notas:") 
        Utils.print_lista_de_notas(notas_que_tenho)
        print(f" pesos:")
        Utils.print_pesos_de_notas(notas_que_quero)
        print(f", e média {media_desejada} uma combinação de notas possíveis é:")
        Utils.print_lista_de_notas(notas_possiveis)
        print(f"\nO algorítmo demorou {t_exec:.5f} segundos para executar.")

In [36]:
# Teste 1
print("\nCaso13 - Teste 1")
P1 = Nota(peso=0.2*0.6, valor=6.0)
T1 = Nota(peso=0.08, valor=6.0)
T2 = Nota(peso=0.08, valor=6.0)

P2 = Nota(peso=0.2*0.6, valor=None)
P3 = Nota(peso=0.3*0.6, valor=None)
T3 = Nota(peso=0.3*0.4, valor=None)
P4 = Nota(peso=0.3*0.6, valor=None)
T4 = Nota(peso=0.3*0.4, valor=None)



media_desejada = 6

l_notas_que_tenho = [P1, T1, T2]

Caso13.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P2, T3, P3, T4, P4], media_desejada=media_desejada)

# Verificação
notas = [
    P1,
    T1,
    T2,
    Nota(peso=P2.peso, valor=6.5),
    Nota(peso=T3.peso, valor=8),
    Nota(peso=P3.peso, valor=5),
    Nota(peso=T4.peso, valor=6),
    Nota(peso=P4.peso, valor=5.5)
]

print("\n",(Utils.media(notas) - media_desejada) <= 0.04)

# Teste 2
print("\nCaso13 - Teste 2")
P1 = Nota(peso=0.2, valor=6.0)
P2 = Nota(peso=0.2, valor=8.0)

P3 = Nota(peso=0.3, valor = None)
P4 = Nota(peso=0.3, valor = None)

media_desejada = 7

l_notas_que_tenho = [P1, P2]

Caso13.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P3, P4], media_desejada=media_desejada)

# Verificação
print("\n"+
str(abs(Utils.media([
    P1,
    P2,
    Nota(peso=P3.peso, valor=7.5),
    Nota(peso=P4.peso, valor=6.5)
]) - media_desejada) <= 0.04)
)

# Teste 3
print("\nCaso13 - Teste 3")
P1 = Nota(peso=0.2, valor=0)
P2 = Nota(peso=0.2, valor=8.0)
P3 = Nota(peso=0.3)
P4 = Nota(peso=0.3)
media_desejada = 10

l_notas_que_tenho = [P1, P2]

Caso13.teste_algoritmo(notas_que_tenho=l_notas_que_tenho, notas_que_quero=[P3, P4], media_desejada=media_desejada)


Caso13 - Teste 1
[5.5, 6.0, 8.0, 9.0, 6.5, 9.5, 7.5, 0.5, 5.0, 3.0, 7.0, 0.0, 10.0, 4.5, 8.5, 4.0, 2.0, 1.5, 3.5, 2.5, 1.0]
[3.5, 10.0, 5.5, 1.0, 5.0, 6.5, 9.5, 4.0, 8.0, 8.5, 3.0, 4.5, 0.0, 0.5, 9.0, 2.0, 1.5, 7.0, 6.0, 2.5, 7.5]
[4.5, 3.0, 0.5, 10.0, 8.5, 0.0, 2.0, 7.0, 8.0, 9.0, 5.0, 6.0, 2.5, 1.0, 6.5, 9.5, 3.5, 1.5, 5.5, 7.5, 4.0]
[4.5, 3.0, 1.5, 9.0, 0.5, 6.0, 10.0, 5.0, 5.5, 7.5, 2.0, 0.0, 8.0, 9.5, 8.5, 6.5, 3.5, 4.0, 1.0, 2.5, 7.0]
[9.0, 1.5, 5.0, 5.5, 2.0, 10.0, 8.0, 3.5, 6.0, 7.5, 8.5, 0.5, 6.5, 1.0, 9.5, 7.0, 4.0, 2.5, 0.0, 3.0, 4.5]
Para as notas:
[ (Valor: 6.0, Peso: 0.12), (Valor: 6.0, Peso: 0.08), (Valor: 6.0, Peso: 0.08) ] pesos:
[ 0.12, 0.12, 0.18, 0.12, 0.18 ], e média 6 uma combinação de notas possíveis é:
[ (Valor: 5.5, Peso: 0.12), (Valor: 3.5, Peso: 0.12), (Valor: 4.5, Peso: 0.18), (Valor: 8.0, Peso: 0.12), (Valor: 8.0, Peso: 0.18) ]
O algorítmo demorou 0.00803 segundos para executar.

 True

Caso13 - Teste 2
[5.0, 9.0, 4.5, 10.0, 9.5, 5.5, 7.0, 6.0, 4.0, 6.5, 8